Problem 1:



In [0]:
# You have `n` examples. Each example has `m` elements of class `Artist`.
# class Artist has the following fields:

import pandas as pd 
pd.options.display.max_rows = 4000

class Artist:
    def __init__(self, name, popularity):
        self.name = name
        self.popularity = popularity

    def __repr__(self):
        return '{} (popularity: {})'.format(self.name, self.popularity)
#
# Assume you don't want to include in your training data very obscure (unpopular) artists.
# How you would go about cleaning your data? What is the complexity of this operation?
# For example assume you have the list of artists:
a = [Artist('Madonna', 1000), Artist('Lady Gaga', 1500), Artist('Rolling Stones', 2500),
     Artist('Metallica', 900), Artist('The Beatles', 5000), Artist('Miley Cyrus', 800)]
#
# and a population threshold: `popularity_threshold` = 1000
# That means, you do not want artists whose population is less than 1000.
# Your training data looks like this (just three rows here for convenience):
df = pd.DataFrame({'user_sentiment': [[a[1], a[2], a[5], a[4]],
                                      [a[3], a[1], a[3]],
                                      [a[2], a[1]]]})
# print(df)
# Out[48]:
#     user_sentiment
# 0  [Lady Gaga (1500), Rolling Stones (2500), Miley Cyrus (800), The Beatles (5000)]
# 1  [Metallica (900), Lady Gaga (1500), Metallica (900)]
# 2  [Rolling Stones (2500), Lady Gaga (1500)]
#
# We want that:
# 0  [Lady Gaga (1500), Rolling Stones (2500), The Beatles (5000)]
# 1  [Lady Gaga (1500)]
# 2  [Rolling Stones (2500), Lady Gaga (1500)]

# steps
# filter only by popularity >999
# create new df with new list

df



,user_sentiment
0,"[Lady Gaga (popularity: 1500), Rolling Stones ..."
1,"[Metallica (popularity: 900), Lady Gaga (popul..."
2,"[Rolling Stones (popularity: 2500), Lady Gaga ..."


In [0]:
df_out = pd.DataFrame(columns = ['user_sentiment'])
l2 = []
for idx, x in enumerate(df['user_sentiment']):
    l1 = x
    for y in l1:
        if y.popularity > 900:
            l2.append(y)
            df_out = df_out.append(pd.Series(str(l2), index = df_out.columns), ignore_index=True)

df_out            
            

,user_sentiment
0,[Lady Gaga (popularity: 1500)]
1,"[Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500)]"
2,"[Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500), The Beatles (popularity: 5000)]"
3,"[Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500), The Beatles (popularity: 5000), Lady Gaga (popularity: 1500)]"
4,"[Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500), The Beatles (popularity: 5000), Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500)]"
5,"[Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500), The Beatles (popularity: 5000), Lady Gaga (popularity: 1500), Rolling Stones (popularity: 2500), Lady Gaga (popularity: 1500)]"


Problem Solving Analysis:

This was difficult becuase the output is a list, instead of a discrete value that can be neatly put into a column. I had to iterate through the rows, and then put into a list. Once I did that, I had to iterate through the list, and take only the values based on the condition. Then I had to take the new list, and put it into the new `df_out` as a single value in the column.  

The solution works, and you can test it as needed.  

# Problem 2

In [0]:
# We have noticed that many TasteDive users might mistype the name of the artist they want to find recs for
# and because the system cannot find it, they enter themselves a noisy version of it.
# In our effort to avoid this, we cache the 1000 most popular artist names and we compare them with the entered name.
# Write the following function:
def exists(name, popular_names):
#
# that returns True if `name` is in one of `popular names`, False otherwise.
# Ignore spaces in the names and allow up to two insertions, or up to two deletions or
# up to one insertion and deletion in the entered name.
#
# For example, assuming 'Madonna' is in `popular_names` the following should return True:
# name = ‘Madonna
# name = ‘Mad o n n a’ (spaces are ignored)
# name = ‘Mdonna’ (one deletion)
# name = ‘Maadonnna’ (two insertions)
#
# But the following must return False
# name = ‘Mdnnna’ (two deletions and one insertion)


In [0]:
# Data setup
i1 = 'Madonna'
i2 = 'Mad o n n a'
i3 = 'Mdonna'
i4 = 'Maadonnna'
i5 = 'Mdnnna'
i6 = 'Micheal Jackson'
i7 = 'apple'
i8 = 'Mdnnnaie'
popular_names = 'Madonna', 'Micheal Jackson'

names = i1, i2, i3, i4, i5, i6, i7, i8

In [0]:
def exists(name, popular_names):
    from collections import Counter
    # exact match
    matches = [i for i in popular_names if name in i] 
    if matches:
        return 'True'
    else:
        # match with space
        matches = [i for i in popular_names if 
                   name.replace(' ', '') in i] 
        if matches:
            return 'True'
        else:
            matches = [i for i in popular_names if set(name) == set(i) and len(name) >= len(i)+2]
            if matches:
                return 'True'
            else:
                matches = [i for i in popular_names if len(name) == (len(i)-1) and set(name).issubset(i)]
                if matches:
                    mn = pd.merge(pd.DataFrame([Counter(matches[0])]), pd.DataFrame([Counter(name)]), how='outer')
                    val = mn.iloc[0] - mn.iloc[1]
                    for x in val:
                        if x < 0:
                            return 'False'
                    else:
                        return 'True'
        return 'False'

In [0]:
for x in names:
    print(exists(x, popular_names))

True
True
True
True
False
True
False
False


For this solution, the first two cases are easy to solve for, namely the exact match and the spaces included match. 

For the next set of decisions, I used list comprehension to eliminate the instances where there are double letters. 

The most complex part is deciphering between the valid input `Mdonna` and the invalid input `Mdnnna`. To distinguish between these two, I used the `Counter()` function to create a df of the value from the `popular_names` list and the value from the input. I then subtracted these two. The key understanding is that the invalid scenario's output will always yeild a negative number. I use that as a condition to flag and then return the 'False' flag. 